# Daily Filtering

In [3]:
from nselib import capital_market
import concurrent.futures
from tqdm import tqdm
import warnings
import numpy as np
import pandas as pd
from datetime import datetime
from nse import nse_self
nse = nse_self()
warnings.filterwarnings("ignore")

In [4]:
today = '04-07-2025'

In [5]:
today_stocks_info = nse.get_nse_equities_by_date(today)
stocks = today_stocks_info[today_stocks_info['SERIES'] == 'EQ']['SYMBOL'].tolist()

In [6]:
today_stocks_info.loc[
    today_stocks_info['SYMBOL'] == 'SSWL', ['SYMBOL', 'OPEN_PRICE', 'HIGH_PRICE', 'LOW_PRICE', 'CLOSE_PRICE','TTL_TRD_QNTY']
    ].rename(columns={
        'OPEN_PRICE': 'O',
        'HIGH_PRICE':'H',
        'LOW_PRICE': 'L',
        'CLOSE_PRICE': 'C',
        'TTL_TRD_QNTY': 'Volume'
    })

,SYMBOL,O,H,L,C,Volume
2480,SSWL,251.3,253.9,250.35,252.9,164637


In [7]:
nse.get_nse_stock_by_duration('CUB', '1W')

,Symbol,Series,Date,PrevClose,OpenPrice,HighPrice,LowPrice,LastPrice,ClosePrice,AveragePrice,TotalTradedQuantity,TurnoverInRs,No.ofTrades,DeliverableQty,%DlyQttoTradedQty,"ï»¿""Symbol"""
0,NaN,EQ,27-Jun-2025,205.13,206.80,206.80,202.42,204.92,205.81,204.84,1677328,3.435828e+08,34056,1069733,63.78,CUB
1,NaN,EQ,30-Jun-2025,205.81,208.50,221.00,205.10,217.50,218.71,214.77,7214554,1.549471e+09,76077,3596754,49.85,CUB
2,NaN,EQ,01-Jul-2025,218.71,215.00,232.36,214.00,228.90,230.51,224.85,8373774,1.882872e+09,113658,3597799,42.97,CUB
3,NaN,EQ,02-Jul-2025,230.51,226.10,232.55,219.50,223.30,221.44,226.00,6649580,1.502823e+09,80458,1970256,29.63,CUB
4,NaN,EQ,03-Jul-2025,221.44,222.00,223.48,216.10,216.53,216.89,218.78,2845297,6.225026e+08,44824,1407505,49.47,CUB
5,NaN,EQ,04-Jul-2025,216.89,216.88,220.99,211.90,216.82,217.27,216.36,2116720,4.579670e+08,37531,868656,41.04,CUB


# level 1 - MA 200 & Vol 2.25x

In [8]:
filtered_stocks_1 = []

def process_stock(stock):
    try:
        data = nse.get_nse_stock_by_duration(symbol=stock, period='1Y')
    except:
        # skipping; stock with trades less than 1000 TDs
        return None


    # -- MA 200 -- 
    MA = 200
    close_series = data[::-1]['ClosePrice'][:MA]
    if close_series.empty or len(close_series.dropna()) == 0:
        return None 
    
    close = close_series.replace({',': ''}, regex=True).astype(float)
    if len(close) < MA:
        return None

    closePrice = close.iloc[0]
    MA_200 = close.mean()

    # TODO: considering adding some buffer
    if closePrice < MA_200:
        # NOTE: skipping; stock has been listed for less than 200 days
        return None


    # -- Volume > 2.25x 10D Avg --
    try:
        volume_data = capital_market.deliverable_position_data(stock, period='1M')['TradedQty']
        volume = volume_data[::-1].str.replace(',', '', regex=False).astype(int)
    except:
        return None

    current_vol = volume.iloc[0]
    # volume needs to above 100k
    if current_vol > 100_000:
        last_10_days_avg = volume.iloc[1:11].mean()
        if current_vol > last_10_days_avg * 2.25:
            return stock

    return None

print(f'Total no. of stocks: {len(stocks)}')
print()

with concurrent.futures.ThreadPoolExecutor(max_workers=11) as executor:
    futures = [executor.submit(process_stock, stock) for stock in stocks]
    for f in tqdm(concurrent.futures.as_completed(futures), total=len(futures), desc='Flitering'):
        result = f.result()
        if result:
            filtered_stocks_1.append(result)

print(f'Filtered stocks: {len(filtered_stocks_1)}')

Total no. of stocks: 2038



Flitering: 100%|██████████| 2038/2038 [12:23<00:00,  2.74it/s]

Filtered stocks: 64


# level 2 - excluding ETFs

In [9]:
exclusions = ["ETF", "GOLD", "SILVER", "BEES", "NIFTY"]
excluded_stocks = []
filtered_stocks_2 = []

for stk in filtered_stocks_1:
    if any(excl in stk for excl in exclusions):
        excluded_stocks.append(stk)
    else:
        filtered_stocks_2.append(stk)

print(f"Excluded stocks: {excluded_stocks}")
len(filtered_stocks_2)

Excluded stocks: ['ALPL30IETF', 'JUNIORBEES', 'LIQUIDBEES', 'LIQUIDETF', 'LIQUIDBETF']


59

# level 3 - higher opens & higher closes

In [10]:
filtered_stocks_3 = []

def check_price_movement(stock):
    try:
        # get lastest 7 days of data
        data = nse.get_nse_stock_by_duration(symbol=stock, period='1W')
        # latest
        data = data[::-1]  
        close_prices = data['ClosePrice'].replace({',': ''}, regex=True).astype(float)
        open_prices = data['OpenPrice'].replace({',': ''}, regex=True).astype(float)

        if len(close_prices) >= 2 and len(open_prices) >= 1:
            today_close = close_prices.iloc[0]
            prev_close = close_prices.iloc[1]
            today_open = open_prices.iloc[0]
            prev_open = open_prices.iloc[1]

            # Condition: today's close > previous close AND today's close > open
            if today_close > prev_close and today_close > today_open and today_open > prev_open:
                return stock
    except Exception as e:
        return None
    return None


with concurrent.futures.ThreadPoolExecutor(max_workers=8) as executor:
    futures = [executor.submit(check_price_movement, stock) for stock in filtered_stocks_2]
    for f in tqdm(concurrent.futures.as_completed(futures), total=len(futures), desc='Price Check'):
        result = f.result()
        if result:
            filtered_stocks_3.append(result)

# sorting alphabetically
filtered_stocks_3.sort()
print(f"Filtered stocks: {len(filtered_stocks_3)}")
print(filtered_stocks_3)

Price Check: 100%|██████████| 59/59 [00:03<00:00, 18.61it/s]

Filtered stocks: 32
['ACI', 'ANDHRAPAP', 'ASALCBR', 'BODALCHEM', 'BOSCHLTD', 'CARERATING', 'CHENNPETRO', 'CHOICEIN', 'CREDITACC', 'DEVYANI', 'EIMCOELECO', 'ENGINERSIN', 'HCG', 'JGCHEM', 'MANAKALUCO', 'MRPL', 'NATHBIOGEN', 'PARSVNATH', 'PRIMESECU', 'RESPONIND', 'RUCHIRA', 'SAKSOFT', 'SAPPHIRE', 'SHARDACROP', 'SINDHUTRAD', 'SMLISUZU', 'SURYALAXMI', 'TNPL', 'UNIPARTS', 'VASWANI', 'WSTCSTPAPR', 'XPROINDIA']


# level 4 - previous 10 days's daily vol > 100k

In [11]:
filtered_stocks_4 = []

def check_volume_consistency(stock):
    try:
        # Get last 15 days of data (to ensure we have at least 10 trading days)
        volume_data = nse.get_nse_stock_by_duration(symbol=stock, period='1M')['TotalTradedQuantity']
        volume = volume_data[::-1]

        # Take last 10 trading days
        last_10_vols = volume.iloc[:10]

        # Check if all volumes > 100k
        if len(last_10_vols) < 10:
            return None

        if all(v > 100_000 for v in last_10_vols):
            return stock
    except:
        return None
    return None


with concurrent.futures.ThreadPoolExecutor(max_workers=8) as executor:
    futures = [executor.submit(check_volume_consistency, stock) for stock in filtered_stocks_3]
    for f in tqdm(concurrent.futures.as_completed(futures), total=len(futures), desc='Volume Consistency'):
        result = f.result()
        if result:
            filtered_stocks_4.append(result)

# Sort alphabetically
filtered_stocks_4.sort()
print(f"Filtered Stocks_4: {len(filtered_stocks_4)}")
print(filtered_stocks_4)

Volume Consistency: 100%|██████████| 32/32 [00:02<00:00, 12.10it/s]

Filtered Stocks_4: 13
['ANDHRAPAP', 'BODALCHEM', 'CHENNPETRO', 'CHOICEIN', 'CREDITACC', 'DEVYANI', 'ENGINERSIN', 'HCG', 'MRPL', 'PARSVNATH', 'SAKSOFT', 'SHARDACROP', 'SINDHUTRAD']


# -- WIP -- (to be level 5)

# final

In [12]:
filtered_stocks_4.sort()
np.array(filtered_stocks_4)

array(['ANDHRAPAP', 'BODALCHEM', 'CHENNPETRO', 'CHOICEIN', 'CREDITACC',
       'DEVYANI', 'ENGINERSIN', 'HCG', 'MRPL', 'PARSVNATH', 'SAKSOFT',
       'SHARDACROP', 'SINDHUTRAD'], dtype='<U10')

# prompt - TODO: RSI & MACD

In [27]:

    hist_data = nse.get_nse_stock_by_duration(symbol='ANDHRAPAP', period='1Y')[::-1]
    closes = hist_data['ClosePrice'].replace({',': ''}, regex=True).astype(float)
    highs = hist_data['HighPrice'].replace({',': ''}, regex=True).astype(float)

In [29]:
highs

248     90.00
247     84.51
246     84.60
245     83.84
244     84.55
        ...  
4      591.00
3      601.45
2      611.60
1      615.65
0      621.00
Name: HighPrice, Length: 249, dtype: float64

In [22]:
def generate_prompt(stk_symbol, today_stocks_info):
    ohlc = today_stocks_info.loc[today_stocks_info['SYMBOL'] == stk_symbol,
                                ['SYMBOL', 'OPEN_PRICE', 'HIGH_PRICE', 'LOW_PRICE', 'CLOSE_PRICE']
    ].values.tolist()[0][1:]

    hist_data = nse.get_nse_stock_by_duration(symbol=stk_symbol, period='1Y')[::-1]
    closes = hist_data['ClosePrice'].replace({',': ''}, regex=True).astype(float)
    highs = hist_data['HighPrice'].replace({',': ''}, regex=True).astype(float)

    ma20 = closes.iloc[:20].mean()
    ma50 = closes.iloc[:50].mean()
    ma200 = closes.iloc[:200].mean()
    today_close = closes.iloc[0]
    ma20_gap = round(((today_close - ma20) / ma20) * 100, 2)
    ma50_gap = round(((today_close - ma50) / ma50) * 100, 2)
    ma200_gap = round(((today_close - ma200) / ma200) * 100, 2)

    yesterday_close = closes.iloc[1]
    today_close = closes.iloc[0]
    change_pct = ((today_close - yesterday_close) / yesterday_close) * 100

    high_52 = highs.max()
    high_gap = round(((high_52 - today_close) / high_52) * 100, 2)


    return f"""
        I'm evaluating **NSE:{stk_symbol}** for a potential **swing buy position**.  
        Please assess this setup based on my strategy context and your expert technical insights.
        Kindly provide a **combined score (1 to 10)** reflecting your preference for a buy at this point also considering the below details.
        

        📊 Stock Details:  
        - Market closed today's 1D chart with:  
        **Open**: {ohlc[0]} | **High**: {ohlc[1]} | **Low**: {ohlc[2]} | **Close**: {ohlc[3]} | **Change**: {change_pct:+.2f}%
        - **20 MA**: {ma20:.2f} (Gap: {ma20_gap:+.2f}%)  
        - **50 MA**: {ma50:.2f} (Gap: {ma50_gap:+.2f}%)  
        - **200 MA**: {ma200:.2f} (Gap: {ma200_gap:+.2f}%)  
        - **RSI**: to_be_filled
        - **MACD** to_be_filled  
        - Gap from 52-week high (%): {high_52} {high_gap:+.2f}%
        - Volume is **2.25x or more** above the average of the last 10 trading days.
        - Both **today's** and **yesterday's** candles show **higher opens and higher closes**.


        📌 **Additional Info (Please fill):**  
        - Earnings in: (Date) | (Gap in weeks)  
        - Sector outlook:  
        - Sector strength: Good / Bad  
        - News related to this stock: Good / Bad  
         
        - Institutional investor interest:  
        - Analysts' research:  
        - Latest earnings: Miss / Beat estimates?

        ---

        🧾 **Summary**  
        ✅ Good:  
        ⚠️ Neutral / Caution:  
        ❌ Bad:

        📍 **Ideal Entry Zone** (Now or wait):  
        🎯 **Final Verdict (Score out of 10)**:  
        """
    

In [23]:
filtered_stocks_4[0]

'ANDHRAPAP'

In [24]:
a = generate_prompt(filtered_stocks_4[0], today_stocks_info)
print(a)


        I'm evaluating **NSE:ANDHRAPAP** for a potential **swing buy position**.  
        Please assess this setup based on my strategy context and your expert technical insights.
        Kindly provide a **combined score (1 to 10)** reflecting your preference for a buy at this point also considering the below details.
        

        📊 Stock Details:  
        - Market closed today's 1D chart with:  
        **Open**: 84.0 | **High**: 90.0 | **Low**: 83.3 | **Close**: 88.18 | **Change**: +4.98%
        - **20 MA**: 80.92 (Gap: +8.97%)  
        - **50 MA**: 78.23 (Gap: +12.72%)  
        - **200 MA**: 86.79 (Gap: +1.60%)  
        - **RSI**: to_be_filled
        - **MACD** to_be_filled  
        - Gap from 52-week high (%): 621.0 +85.80%
        - Volume is **2.25x or more** above the average of the last 10 trading days.
        - Both **today's** and **yesterday's** candles show **higher opens and higher closes**.


        📌 **Additional Info (Please fill):**  
        - Earning

# final_stocks prices

In [17]:
for stk in filtered_stocks_4:
    result = today_stocks_info.loc[
        today_stocks_info['SYMBOL'] == stk,
        ['SYMBOL', 'OPEN_PRICE', 'HIGH_PRICE', 'LOW_PRICE', 'CLOSE_PRICE']
    ]
    
    print(result.values.tolist())
    print()

[['AFFLE', 1979.0, 2080.0, 1975.1, 2044.8]]

[['ASTERDM', 595.5, 675.0, 595.45, 650.05]]

[['BLKASHYAP', 66.1, 73.45, 65.88, 70.65]]

[['KRBL', 380.0, 398.0, 377.55, 396.8]]

[['LATENTVIEW', 425.0, 450.0, 420.0, 446.7]]

[['MRPL', 144.3, 151.4, 144.1, 147.62]]

[['MSUMI', 59.69, 61.75, 59.5, 60.38]]

[['SAKSOFT', 201.75, 235.0, 201.75, 225.87]]

[['SHARDACROP', 848.9, 921.9, 840.6, 886.9]]

[['UDAICEMENT', 34.6, 35.6, 34.43, 34.83]]



# 9.15 to 9.30

In [ ]:
import requests
import time
from datetime import datetime

In [ ]:
api_key = '2dabbfcdf93040ba99894559f7a27ac5'
symbol = 'RELIANCE.NSE'

base_url = 'https://api.twelvedata.com/time_series'

# time window: 9.15 to 9.30
start_hour = 9
start_minute = 15
end_minute = 30

while True:
    # current time
    now = datetime.now()
    
    # if inside the window
    if now.hour == start_hour and start_minute <= now.minute <= end_minute:
        params = {
            'symbol': symbol,
            'interval': '5min',
            'outputsize': 1,
            'apikey': api_key
        }
        response = requests.get(base_url, params=params)
        data = response.json()

        if 'values' in data:
            last_candle = data['values'][0]
            print(f"{last_candle['datetime']} - O:{last_candle['open']} H:{last_candle['high']} L:{last_candle['low']} C:{last_candle['close']}")
        else:
            print("Error or limit reached:", data)

        time.sleep(300)  # Wait 5 mins

    elif now.minute > end_minute:
        print("Finished 9:15–9:30 fetching.")
        break
    else:
        time.sleep(10)  # Before 9:15, check every 10 sec
